# pseudocode

## 1. Word probability

In [1]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

## 2.Write a next-state function

In [2]:
letters    = 'abcdefghijklmnopqrstuvwxyz'
# state 是一個tuple
def next_states(state):
    # appearant
    L, R, edit, prob = state
    R0, R1 = R[0], R[1:]
    if edit == 2: return [(L+R0,R1,edit,prob)]
    # ('a', 'ppearant', 0, 0.0)
    noedit = [(L+R0, R1, edit, prob)]
    # ('', 'ppearant', 1, 0.0) 
    delete = [(L, R1, edit + 1, P((L + R1)))]
    #('a', 'ppearant', 1, 0.0)
    replace = [(L + c, R1, edit + 1, P((L + c + R1))) for c in letters]
    #('aa', 'ppearant', 1, 0.0)          
    insert  = [(L + R0 + c, R1, edit + 1, P((L + R0 + c + R1))) for c in letters]
    return noedit + delete + replace + insert 


### Combine states with the same L and R so the edit is minimized

In [3]:
def combine_L_R(states):
    word_tuple = {}
    for L, R, edit, Prob in states:
        word = L + R
        if word not in word_tuple:
            word_tuple[word] = (L, R, edit, Prob)
#         elif Prob < word_tuple[word][2]:
#             word_tuple[word] = (L, R, Prob)
    return list(word_tuple.values())  
# pprint(combine_L_R(next_states(('', 'appearant', 0, P('appearant')))))

### Sort the states in STATES in increasing values of edit  then decreasing probability of state

In [4]:
from operator import itemgetter

def sort_state(states):
    sorted_by_prob = sorted(states,key=itemgetter(3),reverse = True)
    return sorted(sorted_by_prob,key=itemgetter(2))

### Filter STATES and keep only states with edit==0 or probability>0

In [5]:
def filter_state(states):
    for state in states:
        if (state[2] != 0) or (state[3] <= 0):
            del state 
    return states

In [6]:
MAXBEAM = 500
from pprint import pprint
def correction(word):
    states = [("", word, 0, P(word))]
    for i in range(len(word)):
        # print(i, states[:3])
        states = [ state for states in map(next_states, states) for state in states ]
        states = combine_L_R(states)     
        states = sort_state(states)[:MAXBEAM]
    filter_state(states)
    
    if len(states)>1:
        for s in states:
            if s[2] == 0:
                del s

    states = sorted(states, key=lambda x: x[3], reverse=True)
    return states[:3]
    
pprint(correction('appearant'))

[('appearance', '', 2, 0.00012101274219355764),
 ('apparent', '', 2, 3.764840868244015e-05),
 ('appearing', '', 2, 2.061698570705056e-05)]


In [7]:
pprint(correction('writtung'))

[('written', '', 2, 0.00010487770990108329),
 ('writing', '', 2, 6.185095712115169e-05),
 ('writhing', '', 2, 3.585562731660967e-06)]


In [8]:
pprint(correction('bad'))

[('and', '', 2, 0.03434251984384874),
 ('a', '', 2, 0.018935356785901566),
 ('was', '', 2, 0.010227817692062909)]


In [9]:
pprint(correction('happy'))

[('happy', '', 0, 0.00019541316887552272),
 ('happen', '', 2, 8.874267760860894e-05),
 ('apply', '', 2, 3.85447993653554e-05)]
